# Westeros Tutorial - Extension of GAMS Formulation

This tutorial demonstrates the procedure for changing the GAMS formulation in MESSAGEix. This will be useful for adding new sets, parameters, variables, and equations, to generate a different output or to represent new phenomena, sectors, etc.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

_This tutorial was presented by [Behnam Zakeri](https://iiasa.ac.at/staff/behnam-zakeri) at the **MESSAGEix Community Meeting** May 2022. Please feel free to suggest improvements through issues and pull-requests_.

In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform()

## Part 1: Adding a new variable `INVEST`
In the first part, we change the GAMS code by adding a new equation called `INVESTMENT_EQUIVALENCE`, which calculates the investment per region in each model period as a new variable called `INVEST`.

### 1.1. Modifying GAMS files
For representing the new equation, we do the following modifications in the GAMS file `model_core.gms`:
1. Introduce a new positive variable for "investment per node and year" as: `INVEST(node, year_all)`
2. Introduce a new equation for "investment per node and year" as: `INVESTMENT_EQUIVALENCE`
3. Specify the new equation as:

$$\text{INVEST}_{n,y}
    = \sum_{t} (\text{inv_cost}_{n,t,y} \cdot \text{construction_time_factor}_{n,t,y} \cdot \text{CAP_NEW}_{n,t,y})$$

You can check these changes in `model_core.gms` [here](https://github.com/iiasa/message_ix/pull/602/files).

### 1.2. Solving the scenario after modifications in GAMS:
We clone a scenario from Westeros "baseline" and solve it again with the new changes in the GAMS side and check the outcome.

In [3]:
# Loading the baseline scenario and cloning to a new scenario called "investment"
model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(model, "investment", keep_solution=False)

In [4]:
# Solving the new scenario again (for testing if the changes in the GAMS code have done anything)
scen.solve()

### 1.3. Checking the outcome
Let's compare the objective value of the "baseline" and "investment" scenarios. We use the python method `assert` for doing this assertion. If this equality does not hold true, the assertion method will throw an error. 

In [5]:
# We assert that the objective values of the two scenarios are the same
assert base.var("OBJ")["lvl"] == scen.var("OBJ")["lvl"]

In [6]:
# We can also assert that the activity of one technology is equal in both scenarios
assert base.var("ACT", {"technology": "coal_ppl"})["lvl"].sum() == scen.var("ACT", {"technology": "coal_ppl"})["lvl"].sum()

#### Observation 1: The results are the same
As asserted above, the objective value, i.e., the total cost of the system, as well as the level of activity of "coal_ppl" is the same in both scenarios. This means the modifications we introduced in the GAMS side have not changed the output of the scenario.

#### Observation 2: variable `INVEST` in the output GDX file
If we check the output GDX file of the new scenario, we should be able to see the new variable `INVEST`. This means that the new variable has been calculated as part of the GAMS mathematical model.

Now, let's check if we can fetch the information about the new variable `INVEST`:

In [7]:
# Reading the content variable "INVEST"
scen.var("INVEST")

,node,year,lvl,mrg


#### Observation (3): Python side does not know about the changes in GAMS
As the empty dataframe above shows, the python side, i.e., `message_ix` code is not aware of the changes we have done in the GAMS side. Therefore, we cannot retrieve the content of variable `INVEST` from the GDX file, even though this variable exists there.

### 1.4. Updating the python code based on the changes in GAMS
In the `message_ix` python package, there is a configuration file for specifying and initializing the model items, i.e., sets, parameters, and variables. This file is called [`models.py`](https://github.com/iiasa/message_ix/blob/main/message_ix/models.py).

1. For updating the file for our changes, we introduce the new variable `INVEST` consistent with the format shown in the file, i.e., adding the line below under "MESSAGE_ITEMS"/"Variables":

`"INVEST": item("var", "n y"),`

This notation informs the `message_ix` package that there exists a variable called `INVEST` with the index sets of "n" for "node" and "y" for "year". After adding this information, the python side will initialize this variable for any new or cloned scenario.

2. For being able to retrieve the content of this new variable, we need to explicitly pass it through the `var_list` option when calling `solve()`. This has to be done because this variable is not among the default variables of `message_ix`. So, the notation for solving will be:

`solve(var_list=["INVEST"])`

In [8]:
# Now, let's solve the scenario again
scen = scen.clone(keep_solution=False)
scen.solve(var_list=["INVEST"])

Now, let's check the content of `INVEST`:

In [9]:
scen.var("INVEST")

,node,year,lvl,mrg
0,World,700,0.000000,7.721735
1,World,710,0.000000,4.740475
2,World,720,0.000000,2.910241
3,Westeros,700,2380.580683,0.000000
4,Westeros,710,3432.070015,0.000000
5,Westeros,720,2195.600646,0.000000


### Congratulations!
You were able to modify the GAMS code, add a new variable and equation, update the python code accordingly, and keep your modeling workflow working seamlessly.

## Part II: Adding a new MESSAGEix parameter 
In the second part of this tutorial, we further modify the GAMS code by adding a new parameter. The goal is to understand what modifications should be done when a new parameter is added.

### 2.1. An upper bound on investment
The new parameter will be an upper bound on investment called `bound_investment_up`. This parameter can be defined per technology types, but here for simplicity we define this bound per node and year, similar to variable `INVEST`. We can define this bound as an absolute value, i.e., maximum amount that can be invested, or we can define it relative to the total system costs, e.g., specifying that investment can be up to 80% of total costs. We choose the former.

Adding a parameter in the GAMS side is different from adding variables shown in Part I of this tutorial. Because GAMS expects this parameter as an input data to be passed from the python side. Therefore, the following additional modifications are needed:

#### a. GAMS side

1. Define the new parameter in the GAMS file `parameter_def.gms`
2. Add the new parameter to the items to be loaded from the input GDX file in `data_load.gms`

and as shown in Part I:

3. Introduce a new equation in `model_core.gms` called `INVESTMENT_CONSTRAINT`
4. Add the following equation to `model_core.gms` for the new bound.

$$\text{INVEST}_{n,y} \leq \text{bound_investment_up}_{n,y}$$

You can check these changes [here](https://github.com/iiasa/message_ix/pull/602/files).

#### Notice:
Please note that we filter this equation by `bound_investment_up` using the $ sign in GAMS. This means that if this parameter is not defined in a scenario, this equation will not be in effect. This way, we retain backward compatibility, i.e., we can still solve our scenarios that do not have bound on investment.

#### b. Python side
As mentioned in Part I, we need to update the `message_ix` python code about the new changes. This can be simply done by adding a line of code to the model configuration file (`models.py`) as shown in Section 1.4 above.

5. Add new parameter to the `MESSAGE_ITEMS` in the model configuration file `models.py`:

`"bound_investment_up": item("par", "n y"),`


In [10]:
# Now, let's clone a new scenario from "baseline" and solve
scen2 = base.clone(scenario="investment_bound", keep_solution=False)
scen2.solve(var_list=["INVEST"])

In [11]:
df = scen2.var("INVEST", {"node": "Westeros"})
df

,node,year,lvl,mrg
0,Westeros,700,2380.580683,0.0
1,Westeros,710,3432.070015,0.0
2,Westeros,720,2195.600646,0.0


Everything seems fine. Now, we can add a value to `bound_investment_up` too. The total investment averaged over the entire modeling horizon is around 2667 million \\$ per year (how ?). Let's assume an investment cap of 3000 million $ per year, e.g., to distribute the investment over modeling horizon, and see if the model solves.

In [12]:
# Adding input data for "bound_investment_up"
bound = pd.DataFrame({
    "node": "Westeros",
    "year": [700, 710, 720],
    "value": 3000,
    "unit": "M$",
    })
# Check the input data before adding to the scenario
bound

,node,year,value,unit
0,Westeros,700,3000,M$
1,Westeros,710,3000,M$
2,Westeros,720,3000,M$


In [13]:
# Adding the unit to the platform (if not exists yet)
mp.add_unit("M$")

# Adding data to the scenario
scen2.remove_solution()
scen2.check_out()
scen2.add_par("bound_investment_up", bound)
scen2.add_par("bound_investment_up", bound)
scen2.commit("bound added")

In [14]:
# Solving the scenario
scen2.solve(var_list=["INVEST"])

### 2.2. Checking the results
In this Section, we check the results after adding the bound compared to the case before. We do this comparison on a few outputs, including:
- investment needs
- total system costs
- price of "light"

In [15]:
# Investment needs before the bound
scen.var("INVEST", {"node": "Westeros"})

,node,year,lvl,mrg
3,Westeros,700,2380.580683,0.0
4,Westeros,710,3432.070015,0.0
5,Westeros,720,2195.600646,0.0


In [16]:
# Investment needs after the bound
scen2.var("INVEST", {"node": "Westeros"})

,node,year,lvl,mrg
0,Westeros,700,2812.650698,0.0
1,Westeros,710,3000.000000,0.0
2,Westeros,720,2458.319649,0.0


#### Observation (1): Investment dynamics
The results show that after adding an upper bound on investment, the model has to distribute investment differently, to remain under the maximum threshold. This has resulted in higher investments in 700 and 720 compared to the previous scenario without annual investment bounds. How much has the total investment changed and why?

Let's compare the objective value, i.e., the total system costs

In [17]:
# Difference in the total system costs after investment limits (M$)
scen2.var("OBJ")["lvl"] - scen.var("OBJ")["lvl"]

2052.6875

#### Observation (2): Total system costs
After adding an upper bound on investment, the total costs of the system has increased compared to the previous scenario without investment bounds. More importantly, the increase in total system cost is not only due to an increase in total investment but also a higher O&M costs.

**Question**: how can we calculate the change in O&M cost?

## Exercise
1- Modify the equation `INVESTMENT_EQUIVALENCE` so that the upper bound on investment will be as % of total system costs.

2- How can we relate the upper bound on investment as % of GDP in MESSAGEix?

In [18]:
# Close the connection to the database
mp.close_db()